In [22]:
import multiprocessing
num_cores = multiprocessing.cpu_count() # parallelize training on multiple CPU cores

import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
from gensim.utils import simple_preprocess # tokenizer, ignores punctuation

In [2]:
# load data# load data
df = pd.read_pickle("./df_2020.pkl")
df


,Headline,Date,category,Word count,Article: No Stopwords,Lemmas,BoW
0,Ukrainischer Kleriker warnt vor den Schweinere...,2020-09-28 06:45:00,gesellschaft,8,"[Ukrainischer, Kleriker, warnt, Schweinereien,...","[Ukrainischer, Kleriker, warnen, Schweinerei, ...","{'Ukrainischer': 1, 'Kleriker': 1, 'warnen': 1..."
1,Der MagnetKünstler Kleben und kleben lassen Video,2020-09-27 19:55:00,gesellschaft,7,"[MagnetKünstler, Kleben, kleben, lassen, Video]","[MagnetKünstler, Kleben, kleben, lassen, Video]","{'MagnetKünstler': 1, 'Kleben': 1, 'kleben': 1..."
2,Es grünt Guccis GärtnerJeans mit Grasflecken f...,2020-09-26 16:55:00,gesellschaft,15,"[grünt, Guccis, GärtnerJeans, Grasflecken, kna...","[grünt, Guccis, GärtnerJeans, Grasflecken, kna...","{'grünt': 1, 'Guccis': 1, 'GärtnerJeans': 1, '..."
3,Chef des Weltwirtschaftsforums Neoliberalismus...,2020-09-26 16:20:00,gesellschaft,8,"[Chef, Weltwirtschaftsforums, Neoliberalismus,...","[Chef, Weltwirtschaftsforums, Neoliberalismus,...","{'Chef': 1, 'Weltwirtschaftsforums': 1, 'Neoli..."
4,Umstrittene Maßnahme Absichtliche CoronaInfekt...,2020-09-26 09:41:00,gesellschaft,9,"[Umstrittene, Maßnahme, Absichtliche, CoronaIn...","[Umstrittene, Maßnahme, Absichtliche, CoronaIn...","{'Umstrittene': 1, 'Maßnahme': 1, 'Absichtlich..."
...,...,...,...,...,...,...,...
8561,Nordmazedonien und Griechenland erhalten Gas a...,2020-01-07 06:30:00,wirtschaft,10,"[Nordmazedonien, Griechenland, erhalten, Gas, ...","[Nordmazedonien, Griechenland, erhalten, Gas, ...","{'Nordmazedonien': 1, 'Griechenland': 1, 'erha..."
8562,Paris Finanzminister warnt USA vor Vergeltung ...,2020-01-06 09:52:00,wirtschaft,11,"[Paris, Finanzminister, warnt, USA, Vergeltung...","[Paris, Finanzminister, warnen, USA, Vergeltun...","{'Paris': 1, 'Finanzminister': 1, 'warnen': 1,..."
8563,USMordanschlag auf iranischen General setzt Fi...,2020-01-03 21:34:00,wirtschaft,8,"[USMordanschlag, iranischen, General, setzt, F...","[USMordanschlag, iranisch, General, setzen, Fi...","{'USMordanschlag': 1, 'iranisch': 1, 'General'..."
8564,Indien wird Deutschland als viertgrößte Volksw...,2020-01-03 06:15:00,wirtschaft,11,"[Indien, Deutschland, viertgrößte, Volkswirtsc...","[Indien, Deutschland, viertgrößte, Volkswirtsc...","{'Indien': 1, 'Deutschland': 1, 'viertgrößte':..."


In [3]:
#preprocess 
def preprocess(df_data):
    if 'Video' in df_data:
        df_data_stripped = str(df_data).strip('Video')
        return df_data_stripped
    else:
        return df_data
    
df['Headline'] =  df['Headline'].apply(lambda row: preprocess(row))
         

In [4]:
# build corpus from data
corpus = df['Headline'].to_list()
corpus

['Ukrainischer Kleriker warnt vor den Schweinereien der Transhumanisten',
 'Der MagnetKünstler Kleben und kleben lassen ',
 'Es grünt Guccis GärtnerJeans mit Grasflecken für knapp 700 Euro wird zum Lachobjekt auf Twitter',
 'Chef des Weltwirtschaftsforums Neoliberalismus wegen COVID19Pandemie endgültig erledigt',
 'Umstrittene Maßnahme Absichtliche CoronaInfektionen für ImpfstoffTests in London geplant',
 'Mit BiosensorImplantaten gegen Corona Nicht nur eine Verschwörungstheorie',
 'CoronaAusschuss Wer keine Maske trägt der klaut raubt mordet  Teil 2',
 'Oxfam Das reichste Prozent der Menschen verursacht 15 Prozent der CO2Emissionen',
 'Autor Milosz Matuschek Cancel Culture führt in die intellektuelle Ödnis',
 'Wegen Anonymität Banksy büßt Markenrecht an renommiertem BlumenwerferMotiv ein',
 'ChinaKritiker rufen zu Boykott von Mulan auf',
 '1600 Euro fürs Nichtstun Hamburger Stipendium zieht Bewerber aus aller Welt an',
 'Die Menschen verkümmern vor Einsamkeit zu Tode  Pfarrer prangert

In [5]:
# make training corpus that is tagged with an ID
training_corpus = []

for num, text in enumerate(corpus):
    # annotate documents with integer tags 
    tagged_doc = TaggedDocument(simple_preprocess(text), [num]) 
    # append to list
    training_corpus.append(tagged_doc)
    
training_corpus

[TaggedDocument(words=['ukrainischer', 'kleriker', 'warnt', 'vor', 'den', 'schweinereien', 'der', 'transhumanisten'], tags=[0]),
 TaggedDocument(words=['der', 'magnetkünstler', 'kleben', 'und', 'kleben', 'lassen'], tags=[1]),
 TaggedDocument(words=['es', 'grünt', 'guccis', 'gärtnerjeans', 'mit', 'grasflecken', 'für', 'knapp', 'euro', 'wird', 'zum', 'lachobjekt', 'auf', 'twitter'], tags=[2]),
 TaggedDocument(words=['chef', 'des', 'neoliberalismus', 'wegen', 'covid', 'pandemie', 'endgültig', 'erledigt'], tags=[3]),
 TaggedDocument(words=['umstrittene', 'maßnahme', 'absichtliche', 'für', 'impfstofftests', 'in', 'london', 'geplant'], tags=[4]),
 TaggedDocument(words=['mit', 'gegen', 'corona', 'nicht', 'nur', 'eine'], tags=[5]),
 TaggedDocument(words=['coronaausschuss', 'wer', 'keine', 'maske', 'trägt', 'der', 'klaut', 'raubt', 'mordet', 'teil'], tags=[6]),
 TaggedDocument(words=['oxfam', 'das', 'reichste', 'prozent', 'der', 'menschen', 'verursacht', 'prozent', 'der', 'co', 'emissionen'], t

In [16]:
# train model
model = Doc2Vec(vector_size=100,  # 100-D word and document vectors
                min_count=2, # minimum doc frequency for the vocab
                window=3, # window size
                workers=num_cores, 
                epochs=200) # epochs

# compile vocab
model.build_vocab(training_corpus)

# train
model.train(training_corpus, total_examples=model.corpus_count, epochs=model.epochs)

# save model
model.save("RT_doc2vec.model")

In [18]:
# load model
model= Doc2Vec.load("RT_doc2vec.model")

# print vocab
#model.wv.vocab


In [21]:
model.doesnt_match("virus corona merkel".split())

<ipython-input-21-c7836a94811f>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  model.doesnt_match("virus corona merkel".split())


'merkel'

In [20]:
model.wv.most_similar('deutschland')

[('coronatests', 0.4564287066459656),
 ('meint', 0.41520899534225464),
 ('quarantäne', 0.4145222306251526),
 ('venezuela', 0.39380308985710144),
 ('großstadt', 0.3865569829940796),
 ('tönniesskandal', 0.38110464811325073),
 ('darmstadt', 0.3724313974380493),
 ('serbien', 0.37092405557632446),
 ('den', 0.36431047320365906),
 ('schweiz', 0.3541313111782074)]